Pandas分组聚合 - 高级
===

---

自定义聚合方式
---

在分组聚合的split-apply-combine过程中，apply是核心。Python 本身有高阶函数 apply() 来实现它

    自定义聚合方式：aggregate()，或agg()

之前的聚合方式，所有列只能应用一个相同的聚合函数

#### agg()自定义聚合方式的优势：

    聚合参数是列表
        对数据每列应用多个相同的聚合函数
    聚合参数是字典
        对数据的每列应用一个或多个不同的聚合函数
    聚合参数是自定义函数
        对数据进行一些复杂的操作

自定义聚合方式可以：

    每个列应用不同的聚合方式
    一个列应用多个聚合方式

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({
    'name': ['张三','李四','王五','李四','王五','王五','赵六'],
    'chinese': [18, 53, 67, 63, 59, 70, 94],
    'math': [82, 63, 41, 59, 46, 39, 58],
    'english': [68, 52, 80, 86, 60, 98, 64],
    'test': ['一','一','一','二','二','三','一']
})

df

,name,chinese,math,english,test
0,张三,18,82,68,一
1,李四,53,63,52,一
2,王五,67,41,80,一
3,李四,63,59,86,二
4,王五,59,46,60,二
5,王五,70,39,98,三
6,赵六,94,58,64,一


In [3]:
# 普通分组聚合
df.groupby('name').sum()

,chinese,math,english
name,,,
张三,18,82,68
李四,116,122,138
王五,196,126,238
赵六,94,58,64


In [4]:
# 使用自定义聚合方式实现
df.groupby('name').agg(sum)

,chinese,math,english
name,,,
张三,18,82,68
李四,116,122,138
王五,196,126,238
赵六,94,58,64


聚合参数是列表
---

给每一列同时应用多个聚合函数

In [5]:
df.groupby('name').agg([sum, 'mean', np.min])  # 列表参数函数可以有多种不同写法：直接写函数名（容易出错），函数名写成字符串，ndarray数组函数
# 如果一种写法出错，尝试换其他写法

chinese                 math           english                
         sum       mean amin  sum mean amin     sum       mean amin
name                                                               
张三        18  18.000000   18   82   82   82      68  68.000000   68
李四       116  58.000000   53  122   61   59     138  69.000000   52
王五       196  65.333333   59  126   42   39     238  79.333333   60
赵六        94  94.000000   94   58   58   58      64  64.000000   64

In [6]:
# 将聚合列索引改为自定义方式，元组实现
df.groupby('name')['chinese', 'math'].agg([('求和', sum), ('平均值', 'mean'), ('最小值', min)])

chinese                math        
          求和        平均值 最小值   求和 平均值 最小值
name                                    
张三        18  18.000000  18   82  82  82
李四       116  58.000000  53  122  61  59
王五       196  65.333333  59  126  42  39
赵六        94  94.000000  94   58  58  58

聚合参数是字典
---

每列应用一个不同聚合函数，或者每列应用多个不同的聚合函数

In [7]:
# 语文列聚合函数：求和
df.groupby('name').agg({'chinese': sum})

,chinese
name,
张三,18
李四,116
王五,196
赵六,94


In [8]:
# 语文列聚合函数：求和，平均值
df.groupby('name').agg({'chinese': [sum, 'mean']})

chinese           
         sum       mean
name                   
张三        18  18.000000
李四       116  58.000000
王五       196  65.333333
赵六        94  94.000000

In [9]:
# 选中的多个列，每列都应用不同的多个聚合函数
df.groupby('name').agg({'chinese': [sum, 'mean'], 'math': [np.min, np.max]})

chinese            math     
         sum       mean amin amax
name                             
张三        18  18.000000   82   82
李四       116  58.000000   59   63
王五       196  65.333333   39   46
赵六        94  94.000000   58   58

聚合参数是自定义函数
---

用于一些较为复杂的聚合工作

* 自定义聚合函数要比系统自带的、经过优化的函数慢得多。
* 因为在构造中间分组数据块时存在非常大的开销（函数调用、数据重排等）

In [10]:
df

,name,chinese,math,english,test
0,张三,18,82,68,一
1,李四,53,63,52,一
2,王五,67,41,80,一
3,李四,63,59,86,二
4,王五,59,46,60,二
5,王五,70,39,98,三
6,赵六,94,58,64,一


In [11]:
def aaa(x):
    return x.max() - x.min()

df.groupby('name').agg(aaa)

,chinese,math,english
name,,,
张三,0,0,0
李四,10,4,34
王五,11,7,38
赵六,0,0,0


In [12]:
70-59

11

In [13]:
# 匿名函数实现
df.groupby('name').agg(lambda x: x.max() - x.min())

,chinese,math,english
name,,,
张三,0,0,0
李四,10,4,34
王五,11,7,38
赵六,0,0,0


#### 例：返回 DataFrame 某一列中 n 个最大值

In [14]:
df

,name,chinese,math,english,test
0,张三,18,82,68,一
1,李四,53,63,52,一
2,王五,67,41,80,一
3,李四,63,59,86,二
4,王五,59,46,60,二
5,王五,70,39,98,三
6,赵六,94,58,64,一


In [15]:
# 定一个 top 函数，返回 DataFrame 某一列中 n 个最大值
def top(df, n=2, column='chinese'):
    return df.sort_values(by=column, ascending=False)[:n]

In [16]:
# 将 top 函数用到最原始的数据 (从 csv 中读取出来的)
top(df)
top(df, 2, 'math')  # 修改参数

,name,chinese,math,english,test
0,张三,18,82,68,一
1,李四,53,63,52,一


In [17]:
df.groupby('name').apply(top)

name  chinese  math  english test
name                                    
张三   0   张三       18    82       68    一
李四   3   李四       63    59       86    二
     1   李四       53    63       52    一
王五   5   王五       70    39       98    三
     2   王五       67    41       80    一
赵六   6   赵六       94    58       64    一

In [18]:
# 自定义函数分组聚合参数
df.groupby('name').apply(top, n=1, column='math')

,,name,chinese,math,english,test
name,,,,,,
张三,0,张三,18,82,68,一
李四,1,李四,53,63,52,一
王五,4,王五,59,46,60,二
赵六,6,赵六,94,58,64,一


---
---


其他分组运算（了解）
---

运用 groupby 函数进行分组后，我们能做的事情还有很多，并不局限于聚合汇总

In [19]:
df

,name,chinese,math,english,test
0,张三,18,82,68,一
1,李四,53,63,52,一
2,王五,67,41,80,一
3,李四,63,59,86,二
4,王五,59,46,60,二
5,王五,70,39,98,三
6,赵六,94,58,64,一


#### 过滤数据

例子：输出所有语文考试平均分及格的数据

In [20]:
def bbb(x):
    return x['chinese'].mean() >= 60

df.groupby('name').agg(bbb)  # 返回布尔值

,chinese,math,english,test
name,,,,
张三,False,False,False,False
李四,False,False,False,False
王五,True,True,True,True
赵六,True,True,True,True


In [21]:
df.groupby('name').filter(bbb)

,name,chinese,math,english,test
2,王五,67,41,80,一
4,王五,59,46,60,二
5,王五,70,39,98,三
6,赵六,94,58,64,一


输出所有语文平均分及格的学生

In [22]:
df.groupby('name').filter(bbb).groupby('name').mean()

,chinese,math,english
name,,,
王五,65.333333,42.0,79.333333
赵六,94.000000,58.0,64.000000


#### 使用 transform 函数对所有的数据元素进行转换计算

In [23]:
def ccc(x):
    return x + 10

df.groupby('name').transform(ccc)

,chinese,math,english
0,28,92,78
1,63,73,62
2,77,51,90
3,73,69,96
4,69,56,70
5,80,49,108
6,104,68,74


In [24]:
# 使用向量化运算方式实现
df[['chinese', 'math', 'english']] + 10

,chinese,math,english
0,28,92,78
1,63,73,62
2,77,51,90
3,73,69,96
4,69,56,70
5,80,49,108
6,104,68,74


---

apply是更底层的函数
---

In [25]:
def ccc(x):
    return x + 10

df[['chinese', 'math', 'english']].apply(ccc)  # 效果同上，底层写法，较为繁琐

,chinese,math,english
0,28,92,78
1,63,73,62
2,77,51,90
3,73,69,96
4,69,56,70
5,80,49,108
6,104,68,74


---

例子：每个学生、每科成绩的平均值
---

In [26]:
# 普通分组聚合
df.groupby('name').mean()

,chinese,math,english
name,,,
张三,18.000000,82.0,68.000000
李四,58.000000,61.0,69.000000
王五,65.333333,42.0,79.333333
赵六,94.000000,58.0,64.000000


In [27]:
# apply自定义函数,底层写法，聚合的原理
def ddd(x):
    return x.mean()

df.groupby('name').apply(ddd)

,chinese,math,english
name,,,
张三,18.000000,82.0,68.000000
李四,58.000000,61.0,69.000000
王五,65.333333,42.0,79.333333
赵六,94.000000,58.0,64.000000


例子：将学生某科成绩按由低到高排序，并返回需要的个数
---

    返回语文成绩最低的前三条数据
    返回所有同学语文成绩最低的1次考试成绩
    返回所有同学数学成绩最低的2次考试成绩

In [28]:
df

,name,chinese,math,english,test
0,张三,18,82,68,一
1,李四,53,63,52,一
2,王五,67,41,80,一
3,李四,63,59,86,二
4,王五,59,46,60,二
5,王五,70,39,98,三
6,赵六,94,58,64,一


In [29]:
# 排序知识
a = pd.Series([3,5,1,9,2,4])
a

a.sort_values()  # 升序
a.sort_values(ascending=False)  # 降序

3    9
1    5
5    4
0    3
4    2
2    1
dtype: int64

In [30]:
# DataFrame按列排序

df.sort_values(by='chinese')  # 按语文列进行排序
df.sort_values(by='chinese', ascending=False)

,name,chinese,math,english,test
6,赵六,94,58,64,一
5,王五,70,39,98,三
2,王五,67,41,80,一
3,李四,63,59,86,二
4,王五,59,46,60,二
1,李四,53,63,52,一
0,张三,18,82,68,一


In [31]:
# 返回所有同学语文成绩最低的前三条数据
df.sort_values(by='chinese')[:3]

# 返回数学第一的同学
df.sort_values(by='math', ascending=False)[:1]

,name,chinese,math,english,test
0,张三,18,82,68,一


In [32]:
# 自定义函数实现上面功能，高级
def top(x, p='chinese', n=3, a=True):
    """
    自定义函数实现DataFram对象排序输出功能.
    
    x：传入的DataFrame对象
    n：获取前几个值
    p：按df对象的哪一列排序
    a: 默认True升序，False降序
    """
    return x.sort_values(by=p, ascending=a)[:n]

# top?

# 所有同学语文成绩最低的前3名
top(df)

# 数学倒数第一的同学
top(df, p='math', n=1)

# 英语成绩最高的2位同学
top(df, p='english', n=2, a=False)

,name,chinese,math,english,test
5,王五,70,39,98,三
3,李四,63,59,86,二


使用apply方式调用函数实现

上面是所有数据行操作，下面是分组后的数据操作

In [33]:
df.groupby('name').apply(top)

name  chinese  math  english test
name                                    
张三   0   张三       18    82       68    一
李四   1   李四       53    63       52    一
     3   李四       63    59       86    二
王五   4   王五       59    46       60    二
     2   王五       67    41       80    一
     5   王五       70    39       98    三
赵六   6   赵六       94    58       64    一

In [34]:
# 自定义函数参数设置
# 用于操作的数据表不需要手动传入，如果手动传入会报参数重复错误
df.groupby('name').apply(top, p='math', n=2, a=False)

name  chinese  math  english test
name                                    
张三   0   张三       18    82       68    一
李四   1   李四       53    63       52    一
     3   李四       63    59       86    二
王五   4   王五       59    46       60    二
     2   王五       67    41       80    一
赵六   6   赵六       94    58       64    一

禁止分组键
---

分组键会跟原始对象的索引共同构成结果对象中的层次化索引

将group_keys=False传入groupby即可禁止该效果

In [35]:
df.groupby(['name','test']).sum()
df.groupby(['name','test'], as_index=False).sum()

,name,test,chinese,math,english
0,张三,一,18,82,68
1,李四,一,53,63,52
2,李四,二,63,59,86
3,王五,一,67,41,80
4,王五,三,70,39,98
5,王五,二,59,46,60
6,赵六,一,94,58,64


In [54]:
# 删除，删除分组带来的外层索引
df.groupby('name').apply(top, n=2, p='math')
df.groupby('name', as_index=False).apply(top, n=2, p='math')
df.groupby('name', group_keys=False).apply(top, n=2, p='math')

,name,chinese,math,english,test
0,张三,18,82,68,一
3,李四,63,59,86,二
1,李四,53,63,52,一
5,王五,70,39,98,三
2,王五,67,41,80,一
6,赵六,94,58,64,一


关于groupby调用describe()方法
---

In [55]:
df

,name,chinese,math,english,test
0,张三,18,82,68,一
1,李四,53,63,52,一
2,王五,67,41,80,一
3,李四,63,59,86,二
4,王五,59,46,60,二
5,王五,70,39,98,三
6,赵六,94,58,64,一


In [56]:
df.describe()

,chinese,math,english
count,7.000000,7.000000,7.000000
mean,60.571429,55.428571,72.571429
std,22.824381,14.998413,16.112698
min,18.000000,39.000000,52.000000
25%,56.000000,43.500000,62.000000
50%,63.000000,58.000000,68.000000
75%,68.500000,61.000000,83.000000
max,94.000000,82.000000,98.000000


In [58]:
df['chinese']
df['chinese'].describe()

count     7.000000
mean     60.571429
std      22.824381
min      18.000000
25%      56.000000
50%      63.000000
75%      68.500000
max      94.000000
Name: chinese, dtype: float64

In [59]:
df.groupby('name')['chinese'].mean()

name
张三    18.000000
李四    58.000000
王五    65.333333
赵六    94.000000
Name: chinese, dtype: float64

In [60]:
df.groupby('name')['chinese'].describe()

,count,mean,std,min,25%,50%,75%,max
name,,,,,,,,
张三,1.0,18.000000,NaN,18.0,18.0,18.0,18.0,18.0
李四,2.0,58.000000,7.071068,53.0,55.5,58.0,60.5,63.0
王五,3.0,65.333333,5.686241,59.0,63.0,67.0,68.5,70.0
赵六,1.0,94.000000,NaN,94.0,94.0,94.0,94.0,94.0


In [63]:
df.groupby('name')['chinese'].describe().stack()  # 列转行
df.groupby('name')['chinese'].describe().unstack().unstack()  # 行转列
df.groupby('name')['chinese'].describe().T

name,张三,李四,王五,赵六
count,1.0,2.000000,3.000000,1.0
mean,18.0,58.000000,65.333333,94.0
std,NaN,7.071068,5.686241,NaN
min,18.0,53.000000,59.000000,94.0
25%,18.0,55.500000,63.000000,94.0
50%,18.0,58.000000,67.000000,94.0
75%,18.0,60.500000,68.500000,94.0
max,18.0,63.000000,70.000000,94.0


将DataFrame分组后应用describe()函数

dataframe分组后之所以可以进行describe操作，原因是生成的结果是层次化索引（相当于3维数据）

In [64]:
df.groupby('name').mean()

,chinese,math,english
name,,,
张三,18.000000,82.0,68.000000
李四,58.000000,61.0,69.000000
王五,65.333333,42.0,79.333333
赵六,94.000000,58.0,64.000000


In [65]:
x3 = df.groupby('name').describe()
x3

chinese                                                     math        \
       count       mean       std   min   25%   50%   75%   max count  mean   
name                                                                          
张三       1.0  18.000000       NaN  18.0  18.0  18.0  18.0  18.0   1.0  82.0   
李四       2.0  58.000000  7.071068  53.0  55.5  58.0  60.5  63.0   2.0  61.0   
王五       3.0  65.333333  5.686241  59.0  63.0  67.0  68.5  70.0   3.0  42.0   
赵六       1.0  94.000000       NaN  94.0  94.0  94.0  94.0  94.0   1.0  58.0   

      ...             english                                                \
      ...   75%   max   count       mean        std   min   25%   50%   75%   
name  ...                                                                     
张三    ...  82.0  82.0     1.0  68.000000        NaN  68.0  68.0  68.0  68.0   
李四    ...  62.0  63.0     2.0  69.000000  24.041631  52.0  60.5  69.0  77.5   
王五    ...  43.5  46.0     3.0  79.333333  19.008770  60.0  70.0  80.0  89.0   
赵六    ...  58.0  58.0     1.0  64.000000        NaN  64.0  64.0  64.0  64.0   

            
       max  
name        
张三    68.0  
李四    86.0  
王五    98.0  
赵六    64.0  

[4 rows x 24 columns]

In [67]:
x3['chinese']
x3['chinese'].T

name,张三,李四,王五,赵六
count,1.0,2.000000,3.000000,1.0
mean,18.0,58.000000,65.333333,94.0
std,NaN,7.071068,5.686241,NaN
min,18.0,53.000000,59.000000,94.0
25%,18.0,55.500000,63.000000,94.0
50%,18.0,58.000000,67.000000,94.0
75%,18.0,60.500000,68.500000,94.0
max,18.0,63.000000,70.000000,94.0


In [68]:
x3['chinese']['mean']['王五']

65.33333333333333

In [69]:
x3.stack()  # 列转行
x3.unstack()  # 行转列

                name
chinese  count  张三       1.000000
                李四       2.000000
                王五       3.000000
                赵六       1.000000
         mean   张三      18.000000
                李四      58.000000
                王五      65.333333
                赵六      94.000000
         std    张三            NaN
                李四       7.071068
                王五       5.686241
                赵六            NaN
         min    张三      18.000000
                李四      53.000000
                王五      59.000000
                赵六      94.000000
         25%    张三      18.000000
                李四      55.500000
                王五      63.000000
                赵六      94.000000
         50%    张三      18.000000
                李四      58.000000
                王五      67.000000
                赵六      94.000000
         75%    张三      18.000000
                李四      60.500000
                王五      68.500000
                赵六      94.000000
         max    张三      18.

apply是聚合操作的底层操作

In [71]:
yyy = lambda x: x.describe()

df.groupby('name').apply(yyy)
df.groupby('name').apply(yyy).unstack()  # 行转列

chinese                                                     math        \
       count       mean       std   min   25%   50%   75%   max count  mean   
name                                                                          
张三       1.0  18.000000       NaN  18.0  18.0  18.0  18.0  18.0   1.0  82.0   
李四       2.0  58.000000  7.071068  53.0  55.5  58.0  60.5  63.0   2.0  61.0   
王五       3.0  65.333333  5.686241  59.0  63.0  67.0  68.5  70.0   3.0  42.0   
赵六       1.0  94.000000       NaN  94.0  94.0  94.0  94.0  94.0   1.0  58.0   

      ...             english                                                \
      ...   75%   max   count       mean        std   min   25%   50%   75%   
name  ...                                                                     
张三    ...  82.0  82.0     1.0  68.000000        NaN  68.0  68.0  68.0  68.0   
李四    ...  62.0  63.0     2.0  69.000000  24.041631  52.0  60.5  69.0  77.5   
王五    ...  43.5  46.0     3.0  79.333333  19.008770  60.0  70.0  80.0  89.0   
赵六    ...  58.0  58.0     1.0  64.000000        NaN  64.0  64.0  64.0  64.0   

            
       max  
name        
张三    68.0  
李四    86.0  
王五    98.0  
赵六    64.0  

[4 rows x 24 columns]